# **SpaceX  Falcon 9 first stage Landing Prediction**


# Problem

we will predict if the Falcon 9 first stage will land successfully. SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars; other providers cost upward of 165 million dollars each, much of the savings is because SpaceX can reuse the first stage. Therefore if we can determine if the first stage will land, we can determine the cost of a launch. This information can be used if an alternate company wants to bid against SpaceX for a rocket launch. In this lab, you will collect and make sure the data is in the correct format from an API. The following is an example of a successful and launch.


![](../../Files/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](../../Files/crash.gif)


Most unsuccessful landings are planned. Space X performs a controlled landing in the oceans. 


## Objectives


We will make a get request to the SpaceX API. You will also do some basic data wrangling and formating. 

- Request to the SpaceX API
- Clean the requested data


----


## Setup


We will import the following libraries


In [49]:
import requests
import pandas as pd
import numpy as np
import datetime

# Setting this option will print all collumns of a dataframe
pd.set_option('display.max_columns', None)
# Setting this option will print all of the data in a feature
pd.set_option('display.max_colwidth', None)
# pd.set_option()

Below we will define a series of helper functions that will help us use the API to extract information using identification numbers in the launch data.

From the <code>rocket</code> column we would like to learn the booster name.


In [50]:
# Takes the dataset and uses the rocket column to call the API and append the data to the list
def getBoosterVersion(data):
    for x in data['rocket']:
       if x:
        response = requests.get("https://api.spacexdata.com/v4/rockets/"+str(x)).json()
        BoosterVersion.append(response['name'])

From the <code>launchpad</code> we would like to know the name of the launch site being used, the logitude, and the latitude.


In [51]:
# Takes the dataset and uses the launchpad column to call the API and append the data to the list
def getLaunchSite(data):
    for x in data['launchpad']:
       if x:
         response = requests.get("https://api.spacexdata.com/v4/launchpads/"+str(x)).json()
         Longitude.append(response['longitude'])
         Latitude.append(response['latitude'])
         LaunchSite.append(response['name'])

From the <code>payload</code> we would like to learn the mass of the payload and the orbit that it is going to.


In [52]:
# Takes the dataset and uses the payloads column to call the API and append the data to the lists
def getPayloadData(data):
    for load in data['payloads']:
       if load:
        response = requests.get("https://api.spacexdata.com/v4/payloads/"+load).json()
        PayloadMass.append(response['mass_kg'])
        Orbit.append(response['orbit'])

From <code>cores</code> we would like to learn the outcome of the landing, the type of the landing, number of flights with that core, whether gridfins were used, wheter the core is reused, wheter legs were used, the landing pad used, the block of the core which is a number used to seperate version of cores, the number of times this specific core has been reused, and the serial of the core.


In [53]:
# Takes the dataset and uses the cores column to call the API and append the data to the lists
def getCoreData(data):
    for core in data['cores']:
            if core['core'] != None:
                response = requests.get("https://api.spacexdata.com/v4/cores/"+core['core']).json()
                Block.append(response['block'])
                ReusedCount.append(response['reuse_count'])
                Serial.append(response['serial'])
            else:
                Block.append(None)
                ReusedCount.append(None)
                Serial.append(None)
            Outcome.append(str(core['landing_success'])+' '+str(core['landing_type']))
            Flights.append(core['flight'])
            GridFins.append(core['gridfins'])
            Reused.append(core['reused'])
            Legs.append(core['legs'])
            LandingPad.append(core['landpad'])

Now let's start requesting rocket launch data from SpaceX API with the following URL:


In [54]:
spacex_url="https://api.spacexdata.com/v4/launches/past"

In [55]:
response = requests.get(spacex_url)

Check the content of the response


In [116]:
response.json()[1]['cores']

[{'core': '5e9e289ef35918416a3b2624',
  'flight': 1,
  'gridfins': False,
  'legs': False,
  'reused': False,
  'landing_attempt': False,
  'landing_success': None,
  'landing_type': None,
  'landpad': None}]

You should see the response contains massive information about SpaceX launches. Next, let's try to discover some more relevant information for this project.


# Collecting Data

### Request and parse the SpaceX launch data using the GET request


To make the requested JSON results more consistent, we will use the following static response object for this project:


In [57]:
import pandas as pd
import json

In [58]:
static_json_path = '../../Files/API_call_spacex_api.json'

We now load json file to pandas datafram


In [59]:
#  Read the saved JSON file into a Python object
with open(static_json_path, 'r') as json_file:
    json_data = json.load(json_file)

# Read the saved JSON file into a Pandas DataFrame
data = pd.json_normalize(json_data)

Using the dataframe <code>data</code> print the first 5 rows


In [60]:
# Get the head of the dataframe
data.head()

,static_fire_date_utc,static_fire_date_unix,tbd,net,window,rocket,success,details,crew,ships,capsules,payloads,launchpad,auto_update,failures,flight_number,name,date_utc,date_unix,date_local,date_precision,upcoming,cores,id,fairings.reused,fairings.recovery_attempt,fairings.recovered,fairings.ships,links.patch.small,links.patch.large,links.reddit.campaign,links.reddit.launch,links.reddit.media,links.reddit.recovery,links.flickr.small,links.flickr.original,links.presskit,links.webcast,links.youtube_id,links.article,links.wikipedia,fairings
0,2006-03-17T00:00:00.000Z,1.142554e+09,False,False,0.0,5e9d0d95eda69955f709d1eb,False,Engine failure at 33 seconds and loss of vehicle,[],[],[],[5eb0e4b5b6c3bb0006eeb1e1],5e9e4502f5090995de566f86,True,"[{'time': 33, 'altitude': None, 'reason': 'merlin engine failure'}]",1,FalconSat,2006-03-24T22:30:00.000Z,1143239400,2006-03-25T10:30:00+12:00,hour,False,"[{'core': '5e9e289df35918033d3b2623', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",5eb87cd9ffd86e000604b32a,False,False,False,[],https://images2.imgbox.com/3c/0e/T8iJcSN3_o.png,https://images2.imgbox.com/40/e3/GypSkayF_o.png,None,None,None,None,[],[],None,https://www.youtube.com/watch?v=0a_00nJ_Y88,0a_00nJ_Y88,https://www.space.com/2196-spacex-inaugural-falcon-1-rocket-lost-launch.html,https://en.wikipedia.org/wiki/DemoSat,NaN
1,None,NaN,False,False,0.0,5e9d0d95eda69955f709d1eb,False,"Successful first stage burn and transition to second stage, maximum altitude 289 km, Premature engine shutdown at T+7 min 30 s, Failed to reach orbit, Failed to recover first stage",[],[],[],[5eb0e4b6b6c3bb0006eeb1e2],5e9e4502f5090995de566f86,True,"[{'time': 301, 'altitude': 289, 'reason': 'harmonic oscillation leading to premature engine shutdown'}]",2,DemoSat,2007-03-21T01:10:00.000Z,1174439400,2007-03-21T13:10:00+12:00,hour,False,"[{'core': '5e9e289ef35918416a3b2624', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",5eb87cdaffd86e000604b32b,False,False,False,[],https://images2.imgbox.com/4f/e3/I0lkuJ2e_o.png,https://images2.imgbox.com/be/e7/iNqsqVYM_o.png,None,None,None,None,[],[],None,https://www.youtube.com/watch?v=Lk4zQ2wP-Nc,Lk4zQ2wP-Nc,https://www.space.com/3590-spacex-falcon-1-rocket-fails-reach-orbit.html,https://en.wikipedia.org/wiki/DemoSat,NaN
2,None,NaN,False,False,0.0,5e9d0d95eda69955f709d1eb,False,Residual stage 1 thrust led to collision between stage 1 and stage 2,[],[],[],"[5eb0e4b6b6c3bb0006eeb1e3, 5eb0e4b6b6c3bb0006eeb1e4]",5e9e4502f5090995de566f86,True,"[{'time': 140, 'altitude': 35, 'reason': 'residual stage-1 thrust led to collision between stage 1 and stage 2'}]",3,Trailblazer,2008-08-03T03:34:00.000Z,1217734440,2008-08-03T15:34:00+12:00,hour,False,"[{'core': '5e9e289ef3591814873b2625', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}]",5eb87cdbffd86e000604b32c,False,False,False,[],https://images2.imgbox.com/3d/86/cnu0pan8_o.png,https://images2.imgbox.com/4b/bd/d8UxLh4q_o.png,None,None,None,None,[],[],None,https://www.youtube.com/watch?v=v0w9p3U8860,v0w9p3U8860,http://www.spacex.com/news/2013/02/11/falcon-1-flight-3-mission-summary,https://en.wikipedia.org/wiki/Trailblazer_(satellite),NaN
3,2008-09-20T00:00:00.000Z,1.221869e+09,False,False,0.0,5e9d0d95eda69955f709d1eb,True,"Ratsat was carried to orbit on the first successful orbital launch of any privately funded and developed, liquid-propelled carrier rocket, the SpaceX Falcon 1",[],[],[],[5eb0e4b7b6c3bb0006eeb1e5],5e9e4502f5090995de566f86,True,[],4,RatSat,2008-09-28T23:15:00.000Z,1222643700,2008-09-28T11:15:00+12:00,hour,False,"[{'core': '5e9e289ef3591855dc3b2626', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_succes

In [61]:
data.shape

(107, 42)

In [62]:
data.dtypes

static_fire_date_utc          object
static_fire_date_unix        float64
tbd                             bool
net                             bool
window                       float64
rocket                        object
success                         bool
details                       object
crew                          object
ships                         object
capsules                      object
payloads                      object
launchpad                     object
auto_update                     bool
failures                      object
flight_number                  int64
name                          object
date_utc                      object
date_unix                      int64
date_local                    object
date_precision                object
upcoming                        bool
cores                         object
id                            object
fairings.reused               object
fairings.recovery_attempt     object
fairings.recovered            object
f

In [63]:
data.isnull().sum()

static_fire_date_utc           7
static_fire_date_unix          7
tbd                            0
net                            0
window                         7
rocket                         0
success                        0
details                        7
crew                           0
ships                          0
capsules                       0
payloads                       0
launchpad                      0
auto_update                    0
failures                       0
flight_number                  0
name                           0
date_utc                       0
date_unix                      0
date_local                     0
date_precision                 0
upcoming                       0
cores                          0
id                             0
fairings.reused               33
fairings.recovery_attempt     27
fairings.recovered            36
fairings.ships                24
links.patch.small              0
links.patch.large              0
links.redd

You will notice that a lot of the data are IDs. For example the rocket column has no information about the rocket just an identification number.

We will now use the API to get information about the launches using the IDs given for each launch. Specifically we will be using columns <code>rocket</code>, <code>payloads</code>, <code>launchpad</code>, and <code>cores</code>.


In [115]:
first_record = response.json()[1]

In [65]:
# just a id of rocket is there !!!
first_record['rocket']

'5e9d0d95eda69955f709d1eb'

In [66]:
# just a id here too!!!
first_record['payloads']

['5eb0e4b6b6c3bb0006eeb1e2']

In [67]:
#just like previous :)
first_record['launchpad']

'5e9e4502f5090995de566f86'

In [68]:
# an id and a few details about legs ad reused ....
first_record['cores']

[{'core': '5e9e289ef35918416a3b2624',
  'flight': 1,
  'gridfins': False,
  'legs': False,
  'reused': False,
  'landing_attempt': False,
  'landing_success': None,
  'landing_type': None,
  'landpad': None}]

In [69]:
# len(first_record['cores'])

In [70]:
# with this function we change string or pandas series to a datetime object of pandas
# with dt method we can access to their propertises like date, day, dayofweek , ...
dsample = pd.to_datetime(data['date_utc']).dt.date
# dsample = pd.to_datetime(data['date_utc']).dt.dayofweek
dsample

0      2006-03-24
1      2007-03-21
2      2008-08-03
3      2008-09-28
4      2009-07-13
          ...    
102    2020-10-06
103    2020-10-18
104    2020-10-24
105    2020-11-05
106    2020-11-16
Name: date_utc, Length: 107, dtype: object

In [71]:
# after convert date time to date object we can use datetime librariy in python to restrict or search into that column
dsample2 = dsample[dsample <= datetime.date(2020, 11, 13)]
dsample2

0      2006-03-24
1      2007-03-21
2      2008-08-03
3      2008-09-28
4      2009-07-13
          ...    
101    2020-09-03
102    2020-10-06
103    2020-10-18
104    2020-10-24
105    2020-11-05
Name: date_utc, Length: 106, dtype: object

In [72]:
data['date_utc'].count()

107

In [73]:
# we saw that date_utc does not have null values and has same entity as number of rows in entire data set 
data['date_utc'].isnull().sum(), data['date_utc'].shape

(0, (107,))

### Select Useful Features

In [74]:
# Lets take a subset of our dataframe keeping only the features we want and the flight number, and date_utc.
data = data[['rocket', 'payloads', 'launchpad', 'cores', 'flight_number', 'date_utc']]

# We will remove rows with multiple cores because those are falcon rockets with 2 extra rocket boosters and rows that have multiple payloads in a single rocket.
data = data[data['cores'].map(len)==1]
data = data[data['payloads'].map(len)==1]

# Since payloads and cores are lists of size 1 we will also extract the single value in the list and replace the feature.
data['cores'] = data['cores'].map(lambda x : x[0])
data['payloads'] = data['payloads'].map(lambda x : x[0])

# We also want to convert the date_utc to a datetime datatype and then extracting the date leaving the time
data['date'] = pd.to_datetime(data['date_utc']).dt.date

# Using the date we will restrict the dates of the launches
data = data[data['date'] <= datetime.date(2020, 11, 13)]

In [75]:
data['rocket'].isnull().sum()

0

In [76]:
data['payloads'].isnull().sum()

0

In [77]:
data['launchpad'].isnull().sum()

0

In [78]:
data['cores'].isnull().sum()

0

In [79]:
# we see that non of records have parameter core equal to None
# (data['cores'].map(lambda x:x['core']) == None).value_counts()
# data[data['cores'].map(lambda x:x['core']) == None].value_counts()
data['cores'][data['cores'].map(lambda x:x['core']) == None].value_counts()

Series([], Name: count, dtype: int64)

Here we have some results 

* From the <code>rocket</code> we would like to learn the booster name

* From the <code>payload</code> we would like to learn the mass of the payload and the orbit that it is going to

* From the <code>launchpad</code> we would like to know the name of the launch site being used, the longitude, and the latitude.

* From <code>cores</code> we would like to learn the outcome of the landing, the type of the landing, number of flights with that core, whether gridfins were used, whether the core is reused, whether legs were used, the landing pad used, the block of the core which is a number used to seperate version of cores, the number of times this specific core has been reused, and the serial of the core.

The data from these requests will be stored in lists and will be used to create a new dataframe.


In [80]:
#Global variables 
BoosterVersion = []
PayloadMass = []
Orbit = []
LaunchSite = []
Outcome = []
Flights = []
GridFins = []
Reused = []
Legs = []
LandingPad = []
Block = []
ReusedCount = []
Serial = []
Longitude = []
Latitude = []

Now, let's apply `getBoosterVersion` function method to get the booster version , by calling getBoosterVersion we will fill BoosterVersion variable with version of booster


In [81]:
# Call getBoosterVersion
getBoosterVersion(data)

the list has now been update 


In [82]:
BoosterVersion[0:5]

['Falcon 1', 'Falcon 1', 'Falcon 1', 'Falcon 1', 'Falcon 9']

We can apply the rest of the  functions here:
By calling getLaunchSite we will fill LaunchSite , Latitude, Longitude variables with related value of launch site

In [83]:
# Call getLaunchSite
getLaunchSite(data)

In [84]:
LaunchSite[0:5]

['Kwajalein Atoll',
 'Kwajalein Atoll',
 'Kwajalein Atoll',
 'Kwajalein Atoll',
 'CCSFS SLC 40']

In [85]:
Latitude[0:5]

[9.0477206, 9.0477206, 9.0477206, 9.0477206, 28.5618571]

In [86]:
Longitude[0:5]

[167.7431292, 167.7431292, 167.7431292, 167.7431292, -80.577366]

By calling getPayloadData we will fill Orbit, PayloadMass variables with related values

In [87]:
# Call getPayloadData
getPayloadData(data)

In [88]:
PayloadMass[0:5]

[20, None, 165, 200, None]

In [89]:
Orbit[0:5]

['LEO', 'LEO', 'LEO', 'LEO', 'LEO']

By calling getCoreData we will fill Outcome, Flights, Reused and .. variables with related values

In [90]:
# Call getCoreData
getCoreData(data)

In [91]:
Outcome[0:5]

['None None', 'None None', 'None None', 'None None', 'None None']

In [92]:
Flights[0:5]

[1, 1, 1, 1, 1]

In [93]:
GridFins[0:5]

[False, False, False, False, False]

In [94]:
Reused[0:5]

[False, False, False, False, False]

In [95]:
Legs[0:5]

[False, False, False, False, False]

In [96]:
LandingPad[0:5]

[None, None, None, None, None]

In [97]:
Block[0:5]

[None, None, None, None, 1]

In [98]:
ReusedCount[0:5]

[0, 0, 0, 0, 0]

In [99]:
Serial[0:5]

['Merlin1A', 'Merlin2A', 'Merlin2C', 'Merlin3C', 'B0003']

Finally lets construct our dataset using the data we have obtained. We we **combine** the columns into a dictionary.


In [100]:
launch_dict = {'FlightNumber': list(data['flight_number']),
'Date': list(data['date']),
'BoosterVersion':BoosterVersion,
'PayloadMass':PayloadMass,
'Orbit':Orbit,
'LaunchSite':LaunchSite,
'Outcome':Outcome,
'Flights':Flights,
'GridFins':GridFins,
'Reused':Reused,
'Legs':Legs,
'LandingPad':LandingPad,
'Block':Block,
'ReusedCount':ReusedCount,
'Serial':Serial,
'Longitude': Longitude,
'Latitude': Latitude}


Then, we need to create a Pandas data frame from the dictionary launch_dict.


In [101]:
# Create a data from launch_dict
data = pd.DataFrame(launch_dict)

Show the summary of the dataframe


In [102]:
data.head()

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,1,2006-03-24,Falcon 1,20.0,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,0,Merlin1A,167.743129,9.047721
1,2,2007-03-21,Falcon 1,NaN,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,0,Merlin2A,167.743129,9.047721
2,4,2008-09-28,Falcon 1,165.0,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,0,Merlin2C,167.743129,9.047721
3,5,2009-07-13,Falcon 1,200.0,LEO,Kwajalein Atoll,None None,1,False,False,False,None,NaN,0,Merlin3C,167.743129,9.047721
4,6,2010-06-04,Falcon 9,NaN,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0003,-80.577366,28.561857


# Wrangling and Cleansing Data  


As you can see there are too many `none` values and of course our dataset is mess and we should clean it

### Filter the dataframe to only include `Falcon 9` launches


Finally we will remove the Falcon 1 launches keeping only the Falcon 9 launches. Filter the data dataframe using the <code>BoosterVersion</code> column to only keep the Falcon 9 launches. Save the filtered data to a new dataframe called <code>data_falcon9</code>.


In [103]:
# we see how many unique booster Version we have 
data['BoosterVersion'].value_counts()

BoosterVersion
Falcon 9    90
Falcon 1     4
Name: count, dtype: int64

In [104]:
data_falcon9 = data[data['BoosterVersion'] == 'Falcon 9']

In [105]:
data_falcon9.reset_index(inplace=True, drop=True)

In [106]:
data_falcon9.head()

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,6,2010-06-04,Falcon 9,NaN,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0003,-80.577366,28.561857
1,8,2012-05-22,Falcon 9,525.0,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0005,-80.577366,28.561857
2,10,2013-03-01,Falcon 9,677.0,ISS,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0007,-80.577366,28.561857
3,11,2013-09-29,Falcon 9,500.0,PO,VAFB SLC 4E,False Ocean,1,False,False,False,None,1.0,0,B1003,-120.610829,34.632093
4,12,2013-12-03,Falcon 9,3170.0,GTO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B1004,-80.577366,28.561857


We must rearrange flight number after we removed falcon 1


In [107]:
# use .loc for assign some list to a column instead of directly assign it!!
# data_falcon9['FlightNumber'] = [x for x in range(1, data_falcon9.shape[0]+1)]
data_falcon9.loc[ :, 'FlightNumber'] = [x for x in range(1, data_falcon9.shape[0]+1)]

In [108]:
data_falcon9.head(10)

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,1,2010-06-04,Falcon 9,NaN,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0003,-80.577366,28.561857
1,2,2012-05-22,Falcon 9,525.0,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0005,-80.577366,28.561857
2,3,2013-03-01,Falcon 9,677.0,ISS,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0007,-80.577366,28.561857
3,4,2013-09-29,Falcon 9,500.0,PO,VAFB SLC 4E,False Ocean,1,False,False,False,None,1.0,0,B1003,-120.610829,34.632093
4,5,2013-12-03,Falcon 9,3170.0,GTO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B1004,-80.577366,28.561857
5,6,2014-01-06,Falcon 9,3325.0,GTO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B1005,-80.577366,28.561857
6,7,2014-04-18,Falcon 9,2296.0,ISS,CCSFS SLC 40,True Ocean,1,False,False,True,None,1.0,0,B1006,-80.577366,28.561857
7,8,2014-07-14,Falcon 9,1316.0,LEO,CCSFS SLC 40,True Ocean,1,False,False,True,None,1.0,0,B1007,-80.577366,28.561857
8,9,2014-08-05,Falcon 9,4535.0,GTO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B1008,-80.577366,28.561857
9,10,2014-09-07,Falcon 9,4428.0,GTO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B1011,-80.577366,28.561857


We have removed some values we reset the FlgihtNumber column

### Dealing with Missing Values

We can see below that some of the rows are missing values in our dataset.


In [109]:
data_falcon9.isnull().sum()

FlightNumber       0
Date               0
BoosterVersion     0
PayloadMass        5
Orbit              0
LaunchSite         0
Outcome            0
Flights            0
GridFins           0
Reused             0
Legs               0
LandingPad        26
Block              0
ReusedCount        0
Serial             0
Longitude          0
Latitude           0
dtype: int64

Before we can continue we must deal with these missing values. The <code>LandingPad</code> column will retain None values to represent when landing pads were not used.


Calculate below the mean for the <code>PayloadMass</code> using the <code>.mean()</code>. Then use the mean and the <code>.replace()</code> function to replace `np.nan` values in the data with the mean we calculated.


In [110]:
# Calculate the mean value of PayloadMass column
# we can use both of these ways

# PayloadMass_mean = data_falcon9['PayloadMass'].mean()
PayloadMass_mean = np.mean(data_falcon9['PayloadMass'])

# print(PayloadMass_mean)
# Replace the np.nan values with its mean value
data_falcon9['PayloadMass'].replace(np.nan, PayloadMass_mean, inplace=True)

/tmp/ipykernel_28559/150863331.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_falcon9['PayloadMass'].replace(np.nan, PayloadMass_mean, inplace=True)


In [111]:
# isnull is Alias of isna.
# data_falcon9.isna().sum()
# check NA values, such as None or numpy.NaN
# Characters such as empty strings '' or numpy.inf are not considered NA values 
# (unless you set pandas.options.mode.use_inf_as_na = True).
data_falcon9.isnull().sum()

FlightNumber       0
Date               0
BoosterVersion     0
PayloadMass        0
Orbit              0
LaunchSite         0
Outcome            0
Flights            0
GridFins           0
Reused             0
Legs               0
LandingPad        26
Block              0
ReusedCount        0
Serial             0
Longitude          0
Latitude           0
dtype: int64

Now `PayloadMass` did not have nan values

In [112]:
data_falcon9.dtypes

FlightNumber        int64
Date               object
BoosterVersion     object
PayloadMass       float64
Orbit              object
LaunchSite         object
Outcome            object
Flights             int64
GridFins             bool
Reused               bool
Legs                 bool
LandingPad         object
Block             float64
ReusedCount         int64
Serial             object
Longitude         float64
Latitude          float64
dtype: object

In [113]:
data_falcon9.head(10)

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,1,2010-06-04,Falcon 9,6123.547647,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0003,-80.577366,28.561857
1,2,2012-05-22,Falcon 9,525.000000,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0005,-80.577366,28.561857
2,3,2013-03-01,Falcon 9,677.000000,ISS,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0007,-80.577366,28.561857
3,4,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,None,1.0,0,B1003,-120.610829,34.632093
4,5,2013-12-03,Falcon 9,3170.000000,GTO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B1004,-80.577366,28.561857
5,6,2014-01-06,Falcon 9,3325.000000,GTO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B1005,-80.577366,28.561857
6,7,2014-04-18,Falcon 9,2296.000000,ISS,CCSFS SLC 40,True Ocean,1,False,False,True,None,1.0,0,B1006,-80.577366,28.561857
7,8,2014-07-14,Falcon 9,1316.000000,LEO,CCSFS SLC 40,True Ocean,1,False,False,True,None,1.0,0,B1007,-80.577366,28.561857
8,9,2014-08-05,Falcon 9,4535.000000,GTO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B1008,-80.577366,28.561857
9,10,2014-09-07,Falcon 9,4428.000000,GTO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B1011,-80.577366,28.561857


Now we should have no missing values in our dataset except for in <code>LandingPad</code>.


We can now export it to a <b>CSV</b> for the next section 


In [114]:
# we do not want to save indexes into file
data_falcon9.to_csv('dataset_part_1.csv', index=False)

## Author

<center>Moein (mrpintime)</center>